In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv 
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

In [3]:
path        = 'Traces_Cassandra'
fields      = ['W_95']
X_raw       = pd.read_csv(path+'/X_new_10mbit.csv')
y_raw       = pd.read_csv(path+'/Y_new_10mbit.csv', skipinitialspace=True, usecols=fields)


In [1]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X_raw, y_raw['W_95'], test_size=0.3, random_state=11)
ts = [1, 5, 10, 30]
metrics=list(X_train_full)
idxs = []
for t in ts:
    min_k = 18888999
    min_nmae = 100
    bgGlobal = time.time()
    edGlobal = None
#    print("Para ", t, " segundos: \n")
    X_train = X_train_full[X_train_full.index % t == 0]
    y_train = y_train_full[y_train_full.index % t == 0]
    for k in range(1, 707, 1):
#        print("Executando para k=" + str(k))
        initTime = time.time();
        selector = SelectKBest(f_classif, k=k)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        idxs_selected = selector.get_support(indices=True)
        
        regressor = RandomForestRegressor(max_depth=10, random_state=380, n_estimators=1, bootstrap=False)
        regressor.fit(X_train_new, y_train)
        pred = regressor.predict(X_test_new)
        
        nmae_Selected = (abs(pred - y_test).mean())/y_test.mean()
        endTime = time.time()
#        print("Para k=" + str(k) + " erro= " + str(nmae_Selected * 100) + "% e levou " + str(endTime - initTime) + " segundos.")
        if nmae_Selected < min_nmae:
            min_nmae = nmae_Selected
            min_k = k
            idxs = idxs_selected
    edGlobal = time.time()
    for i in idxs:
        print(metrics[i], end=', ');
    print("\nPara t=" + str(t) + " segundos o menor k=" + str(min_k) + " e nmae=" + str(min_nmae) + " e levou " + str(edGlobal - bgGlobal) + " segundos para completar.")

NameError: name 'train_test_split' is not defined